<a id='0.1'></a>
## 0.1 load required packages

In [1]:
%run "..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

# library design specific tools
from ImageAnalysis3.library_tools import LibraryDesigner as ld
from ImageAnalysis3.library_tools import LibraryTools as lt
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

28492


In [2]:
## Some folders
# human genome
reference_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl'
genome_folder = os.path.join(reference_folder, 'Genome')
# Library directories
pool_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-12_mop_markers'
# folder for sub-pool
library_folder = os.path.join(pool_folder, f'marker_gene_TSS')

In [3]:
# load probes
probes = []
final_pb_filename = os.path.join(library_folder, 'blast_full_probes.fasta')

with open(final_pb_filename, 'r') as handle:
    for record in SeqIO.parse(handle, "fasta"):
        probes.append(record)


In [4]:
from ImageAnalysis3 import library_tools
pb_dict = library_tools.quality_check.split_probe_by_gene(probes)
print(len(pb_dict))

28


In [5]:
# load used readouts
readout_usage_file = os.path.join(library_folder, 'readout_usage.pkl')
readout_dict = pickle.load(open(readout_usage_file, 'rb'))

In [6]:
# select a library
lib_id = 0
# select lib type
lib_type = 'unique'

In [7]:
from ImageAnalysis3.classes import _allowed_kwds
selected_readout_names = [_r.id for _r in readout_dict[_allowed_kwds[lib_type]]]

## bit_2_readout

In [8]:
import pandas as pd

In [9]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
dtype_dict = {_v:_k for _k,_v in ia.classes._allowed_kwds.items()}

In [10]:
for _dtype, _records in readout_dict.items():
    
    bit_readout_df = pd.DataFrame([[_i+1, _r.id] for _i, _r in enumerate(_records)], columns=['Bit','ReadoutName'])
    save_filename = os.path.join(save_folder, f'CTP12-TSS_{dtype_dict[_dtype]}_readouts.csv')
    print(save_filename)
    bit_readout_df.to_csv(save_filename, index=False)

\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-12_mop_markers\Summary_tables\CTP12-TSS_unique_readouts.csv


## Generate sequential codebook

In [11]:
# load region names

gene_2_region = {}

for _pb in probes:
    _reg = _pb.id.split('loc_')[1].split('_')[0]
    _gene = _pb.id.split('gene_')[1].split('_')[0]
    #print(_reg,_gene)
    
    if _gene not in gene_2_region:
        gene_2_region[_gene] = _reg
print(gene_2_region)

{'Aqp4': '18:15400982-15420982', 'Bgn': 'X:73473602-73493602', 'Car3': '3:14853512-14873512', 'Ctss': '3:95516786-95536786', 'Cux2': '5:122040102-122060102', 'Flt1': '5:147716011-147736011', 'Gad1': '2:70543072-70563072', 'Igf2': '7:142656816-142676816', 'Lamp5': '2:136042239-136062239', 'Lratd2': '15:60843778-60863778', 'Nxph4': '10:127524559-127544559', 'Otof': '5:30451932-30471932', 'Pdgfra': '5:75142292-75162292', 'Ptpru': '4:131828288-131848288', 'Pvalb': '15:78196400-78216400', 'Rorb': '19:19101196-19121196', 'Rspo1': '4:124976430-124996430', 'Slc17a7': '7:45153949-45173949', 'Slc30a3': '5:31098237-31118237', 'Slc32a1': '2:158600767-158620767', 'Sncg': '14:34364789-34384789', 'Sox10': '15:79155240-79175240', 'Sst': '16:23880958-23900958', 'Sulf2': '2:166145663-166165663', 'Syt6': '3:103565231-103585231', 'Tshz2': '2:169623013-169643013', 'Vip': '10:5629218-5649218', 'Vtn': '11:78489091-78509091'}


In [12]:
# laod encoding
# summarize total readout usage
gene_2_readout_dict = pickle.load(open(os.path.join(library_folder, 'gene_2_readout.pkl'), 'rb'))
print(gene_2_readout_dict)

{'Slc30a3': ['u0', 'u0'], 'Slc17a7': ['u1', 'u1'], 'Slc32a1': ['u2', 'u2'], 'Gad1': ['u3', 'u3'], 'Otof': ['u4', 'u4'], 'Rspo1': ['u5', 'u5'], 'Pvalb': ['u6', 'u6'], 'Sst': ['u7', 'u7'], 'Vip': ['u8', 'u8'], 'Sncg': ['u9', 'u9'], 'Lamp5': ['u10', 'u10'], 'Lratd2': ['u11', 'u11'], 'Tshz2': ['u12', 'u12'], 'Syt6': ['u13', 'u13'], 'Nxph4': ['u14', 'u14'], 'Cux2': ['u15', 'u15'], 'Rorb': ['u16', 'u16'], 'Sulf2': ['u17', 'u17'], 'Ptpru': ['u18', 'u18'], 'Car3': ['u19', 'u19'], 'Aqp4': ['u20', 'u20'], 'Flt1': ['u21', 'u21'], 'Igf2': ['u22', 'u22'], 'Pdgfra': ['u23', 'u23'], 'Sox10': ['u24', 'u24'], 'Ctss': ['u25', 'u25'], 'Vtn': ['u26', 'u26'], 'Bgn': ['u27', 'u27']}


In [19]:
# generate the sequential_codebook
sequential_codebook = pd.DataFrame(columns=['name','id', 'chr']+[_r.id for _r in readout_dict['u']])
#sequential_codebook.add(['name', 'id'], axis=1)
sequential_codebook['id'] = list(gene_2_readout_dict.keys())
# assign gene and name
sequential_codebook['name'] = [gene_2_region[str(_r)] for _r in list(gene_2_readout_dict.keys())]
sequential_codebook['chr'] = [gene_2_region[str(_r)].split(':')[0] for _r in list(gene_2_readout_dict.keys())]

In [23]:
max_bit = np.max([np.max([int(_b[1:]) for _b in _bits]) for _gname, _bits in gene_2_readout_dict.items()]) + 1
from tqdm.notebook import tqdm
for _gname, _bits in tqdm(gene_2_readout_dict.items()):
    binary_code = []
    for _i in range(max_bit):
        if f"u{_i}" in _bits:
            binary_code.append(1)
        else:
            binary_code.append(0)
    #print(_gname)
    sequential_codebook.loc[sequential_codebook['id']==_gname, sequential_codebook.columns[3:]] = binary_code
    #sequential_codebook.loc[sequential_codebook['name']==_gname,'id'] = int(_bits[0].split('u')[1])
    

  0%|          | 0/28 [00:00<?, ?it/s]

In [24]:
sequential_codebook

,name,id,chr,Stv_91,Stv_32,Stv_92,Stv_33,Stv_94,Stv_35,Stv_95,...,Stv_106,Stv_45,Stv_107,Stv_46,Stv_109,Stv_48,Stv_118,Stv_50,Stv_119,Stv_53
0,5:31098237-31118237,Slc30a3,5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7:45153949-45173949,Slc17a7,7,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2:158600767-158620767,Slc32a1,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2:70543072-70563072,Gad1,2,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5:30451932-30471932,Otof,5,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,4:124976430-124996430,Rspo1,4,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,15:78196400-78216400,Pvalb,15,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,16:23880958-23900958,Sst,16,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,10:5629218-5649218,Vip,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,14:34364789-34384789,Sncg,14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP12-TSS_sequential-codebook.csv')

In [26]:
sequential_codebook.to_csv(save_filename, index=None)

# Design Adaptors

## load readouts

In [11]:
readout_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Readouts'
ref_files = [_fl for _fl in os.listdir(readout_folder) if 'designed_readouts' in _fl]

ref_readout_dict = {}
for _fl in ref_files:
    _channel = int(_fl.split('designed_readouts_')[1].split('.fasta')[0])
    _ref_readout_names = []
    with open(os.path.join(readout_folder, _fl), 'r') as _rd_handle:
        for _readout in SeqIO.parse(_rd_handle, "fasta"):
            _ref_readout_names.append(_readout.id)
    ref_readout_dict[_channel] = _ref_readout_names

In [12]:
# sort regions with readout types
readout_by_channel = {_c:[] for _c in ref_readout_dict}
for _rd in readout_dict[_allowed_kwds[lib_type]]:
    for _c in readout_by_channel:
        if _rd.id in ref_readout_dict[_c]:
            readout_by_channel[_c].append(_rd.id)
readout_by_channel = {_k:_v for _k,_v in sorted(readout_by_channel.items(), key=lambda v:-int(v[0])) if len(_v) > 0}

In [16]:
import csv
lib_name = 'marker-gene-TSS'
num_tubes = 25

save_folder = os.path.join(pool_folder, 'Summary_tables')
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
    
with open(os.path.join(save_folder, f'{lib_name}_{lib_type}_adaptor_sequences.csv'), 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', 
                           quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    _header = ['group', 'hyb', ]
    for _ch in readout_by_channel:
        _header += [f"{_ch}_bits", f"{_ch}_readouts"]
        
    csvwriter.writerow(_header)
    
    for _i in range(max([len(_v) for _k,_v in readout_by_channel.items()])):
        _row = [int((_i)/num_tubes)+1, _i%num_tubes+1,] 
        for _j, (_ch, _names) in enumerate(readout_by_channel.items()):
            if _i >= len(_names):
                _row += ['', '']
            else:
                _row += [f"b{_i*len(readout_by_channel)+_j}", _names[_i]]      
        csvwriter.writerow(_row)

In [17]:
from copy import copy
# find adaptors
reload(library_tools.sequences)
#library_tools.sequences.fasta_reader()

adaptor_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Adaptors'
adaptor_files = [os.path.join(adaptor_folder, _fl) for _fl in os.listdir(adaptor_folder) 
                 if _fl.split(os.extsep)[-1]=='fasta' and _fl.split(os.extsep)[-2][-9:] == '_adaptors']
adaptors = library_tools.sequences.fasta_reader(adaptor_files, True).load()

loading 3 fasta files
- loading from file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Adaptors\20200121_extend_stv_adaptors.fasta
- loading from file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Adaptors\NDB_adaptors.fasta
- loading from file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Adaptors\Stv_adaptors.fasta


# find used adaptors and generate IDT batch order

In [20]:
from copy import copy
selected_adaptors = {}

for _ch, _rnames in readout_by_channel.items():
    selected_adaptors[_ch] = []
    for _rname in _rnames:
        _matched_adaptors = [_adt for _adt in adaptors if _rname in _adt.id]
        if len(_matched_adaptors) == 1:
            _matched_adaptor = copy(_matched_adaptors[0])
            _matched_adaptor.id = _matched_adaptor.id+'rc'
            _matched_adaptor.description = ""

            selected_adaptors[_ch].append(_matched_adaptor)
        else:
            print(_rname)
            
readout_usage_folder = os.path.dirname(readout_usage_file)
# generate csv file to order in IDT
import csv

with open(os.path.join(save_folder, f'{lib_name}_{lib_type}_adaptor_idt_order_tubes.csv'), 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', lineterminator='\n',
                       quotechar='|', quoting=csv.QUOTE_MINIMAL)
    # write header
    _header = ['Name', 'Sequence', 'Scale', 'Purification']
    csvwriter.writerow(_header)
    # write sequence
    for _ch, _adaptors in selected_adaptors.items():
        for _adaptor in _adaptors:
            _info = [_adaptor.id, str(_adaptor.seq), '25nm', 'STD']
            csvwriter.writerow(_info)

# Generate Color_Usage.csv

In [21]:
import csv

drift_channel = '488'
dapi_channel = '405'
ref_hyb = 0

chrom_labels = {}

with open(os.path.join(save_folder, f'{lib_name}_{lib_type}_Color_Usage.csv'), 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', 
                           quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    # write header
    _header = ['Hyb']
    for _ch in readout_by_channel:
        _header .append(str(_ch))
    _header.append(drift_channel)
    _header.append(dapi_channel)
    print(_header)
    csvwriter.writerow(_header)
    
    # write reference frame
    if len(chrom_labels) > 0:
        _ref_row = ['H0R0']
        for _ch in readout_by_channel:
            if str(_ch) in chrom_labels:
                _ref_row.append(chrom_labels[str(_ch)]+'_chrom')
            else:
                _ref_row.append("")
        _ref_row.append('beads')
        _ref_row.append('DAPI')
        print(_ref_row)
        csvwriter.writerow(_ref_row)
    
    
    for _i in range(max([len(_v) for _k,_v in readout_by_channel.items()])):
        _row = [f"H{int(_i)}C{int(_i)+1}",] 
        for _j, (_ch, _names) in enumerate(readout_by_channel.items()):
            if _i >= len(_names):
                _row += ['']
            else:
                _row += [f"{_allowed_kwds[lib_type]}{_i*len(readout_by_channel)+_j}"]
            
        _row.append("beads")
        # append for ref
        if _i == ref_hyb:
            _row.append('DAPI')
        #
        print(_row)
        csvwriter.writerow(_row)


['Hyb', '647', '561', '488', '405']
['H0C1', 'u0', 'u1', 'beads', 'DAPI']
['H1C2', 'u2', 'u3', 'beads']
['H2C3', 'u4', 'u5', 'beads']
['H3C4', 'u6', 'u7', 'beads']
['H4C5', 'u8', 'u9', 'beads']
['H5C6', 'u10', 'u11', 'beads']
['H6C7', 'u12', 'u13', 'beads']
['H7C8', 'u14', 'u15', 'beads']
['H8C9', 'u16', 'u17', 'beads']
['H9C10', 'u18', 'u19', 'beads']
['H10C11', 'u20', 'u21', 'beads']
['H11C12', 'u22', 'u23', 'beads']
['H12C13', 'u24', 'u25', 'beads']
['H13C14', 'u26', 'u27', 'beads']


In [23]:
# Generate color-swap adaptors to change all 561 into 750

# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

In [24]:
# Load readouts
readout_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Readouts'
ref_files = [_fl for _fl in os.listdir(readout_folder) if 'designed_readouts' in _fl]

readout_dict = {}
for _fl in ref_files:
    _channel = int(_fl.split('designed_readouts_')[1].split('.fasta')[0])
    with open(os.path.join(readout_folder, _fl), 'r') as _rd_handle:
        for _readout in SeqIO.parse(_rd_handle, "fasta"):
            readout_dict[_readout.id] = _readout

In [26]:
# load readout sites
adaptor_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Adaptors'
readout_site_file = os.path.join(adaptor_folder, 'Readout_sites.fasta')
readout_sites = []
with open(readout_site_file, 'r') as _rd_handle:
    for _readout in SeqIO.parse(_rd_handle, "fasta"):
        readout_sites.append(_readout)
print(readout_sites)

[SeqRecord(seq=Seq('TTTGCACTGCCGTCCTTGAC'), id='Stv_82rc', name='Stv_82rc', description='Stv_82rc cy7 rev-com_last20', dbxrefs=[]), SeqRecord(seq=Seq('GATCCGATTGGAACCGTCCC'), id='Stv_1rc', name='Stv_1rc', description='Stv_1rc cy5 rev-com_last20', dbxrefs=[]), SeqRecord(seq=Seq('TGCGAACTGTCCGGCTTTCA'), id='Stv_79rc', name='Stv_79rc', description='Stv_79rc cy3 rev-com_last20', dbxrefs=[])]


In [28]:
cy3_names = readout_by_channel[561]
cy3_readouts = [_v for _k,_v in readout_dict.items() if _k in cy3_names]

In [47]:
reload(ia.library_tools.readouts)
from ImageAnalysis3.library_tools.readouts import Generate_adaptors
cy7_for_cy3_adaptors = Generate_adaptors(cy3_readouts, [readout_sites[0]])

In [49]:
cy7_for_cy3_adaptors[0]

SeqRecord(seq=Seq('TGACGTCGGTTTGGACGAGATTTGCACTGCCGTCCTTGACTTTGCACTGCCGTCCTTGAC'), id='Stv_91_2xStv_82rc', name='', description='', dbxrefs=[])

In [50]:
save_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-12_mop_markers\Summary_tables'

# generate csv file to order in IDT
import csv

with open(os.path.join(save_folder, f'{lib_name}-cy7_replace_cy3_adaptors.csv'), 'w') as csvfile:
    csvwriter = csv.writer(csvfile, #delimiter=',', 
                       quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
    # write header
    _header = ['Name', 'Sequence', 'Scale', 'Purification']
    csvwriter.writerow(_header)
    # write sequence
    for _adaptor in cy7_for_cy3_adaptors:
        _info = [_adaptor.id, str(_adaptor.seq), '25nm', 'STD']
        csvwriter.writerow(_info)


In [14]:
gene_2_readouts = pickle.load(open(os.path.join(library_folder, 'gene_2_readout.pkl'), 'rb'))

In [17]:
readout_dict

{'u': [SeqRecord(seq=Seq('TTCCCGCCAATGACGTCGGTTTGGACGAGA'), id='Stv_91', name='Stv_91', description='Stv_91 old_barcode', dbxrefs=[]),
  SeqRecord(seq=Seq('AGACGACGCACGTTCGTACCGCGTACTTCG'), id='Stv_32', name='Stv_32', description='Stv_32 old_barcode', dbxrefs=[]),
  SeqRecord(seq=Seq('GCGTTGATGTCCCTTGTGAGCGCCCGACAT'), id='Stv_92', name='Stv_92', description='Stv_92 old_barcode', dbxrefs=[]),
  SeqRecord(seq=Seq('TTTGCTCGCAAGTGCGCACGAGTTGAACTG'), id='Stv_33', name='Stv_33', description='Stv_33 old_barcode', dbxrefs=[]),
  SeqRecord(seq=Seq('CGCTTATCGATGTCAGGTCCGCATGGGTCG'), id='Stv_94', name='Stv_94', description='Stv_94 old_barcode', dbxrefs=[]),
  SeqRecord(seq=Seq('AGTTTCCATACGCGGCGTGAATATCGCGGC'), id='Stv_35', name='Stv_35', description='Stv_35 old_barcode', dbxrefs=[]),
  SeqRecord(seq=Seq('AACGTCATCGGTGGATCCCAGAGTGCCAAA'), id='Stv_95', name='Stv_95', description='Stv_95 old_barcode', dbxrefs=[]),
  SeqRecord(seq=Seq('GCTACAGCGTTACGGGCCCAGACGTTTCAT'), id='Stv_36', name='Stv_36', de

In [13]:
# load region names

gene_2_region = {}

for _pb in probes:
    _reg = _pb.id.split('loc_')[1].split('_')[0]
    _gene = _pb.id.split('gene_')[1].split('_')[0]
    #print(_reg,_gene)
    
    if _gene not in gene_2_region:
        gene_2_region[_gene] = _reg
        
print(gene_2_region)

{'Aqp4': '18:15400982-15420982', 'Bgn': 'X:73473602-73493602', 'Car3': '3:14853512-14873512', 'Ctss': '3:95516786-95536786', 'Cux2': '5:122040102-122060102', 'Flt1': '5:147716011-147736011', 'Gad1': '2:70543072-70563072', 'Igf2': '7:142656816-142676816', 'Lamp5': '2:136042239-136062239', 'Lratd2': '15:60843778-60863778', 'Nxph4': '10:127524559-127544559', 'Otof': '5:30451932-30471932', 'Pdgfra': '5:75142292-75162292', 'Ptpru': '4:131828288-131848288', 'Pvalb': '15:78196400-78216400', 'Rorb': '19:19101196-19121196', 'Rspo1': '4:124976430-124996430', 'Slc17a7': '7:45153949-45173949', 'Slc30a3': '5:31098237-31118237', 'Slc32a1': '2:158600767-158620767', 'Sncg': '14:34364789-34384789', 'Sox10': '15:79155240-79175240', 'Sst': '16:23880958-23900958', 'Sulf2': '2:166145663-166165663', 'Syt6': '3:103565231-103585231', 'Tshz2': '2:169623013-169643013', 'Vip': '10:5629218-5649218', 'Vtn': '11:78489091-78509091'}


In [18]:
import pandas as pd
# generate the codebooka
codebook = pd.DataFrame(columns=['name','id']+[_r.id for _r in readout_dict['c']])
#codebook.add(['name', 'id'], axis=1)
codebook['id'] = list(region_2_readout_dict.keys())

In [25]:
codebook['name'] = [region_2_names[str(_r)] for _r in list(region_2_readout_dict.keys())]

In [26]:
codebook

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_1028,NDB_925,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910
0,1:3740000-3760000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1:6240000-6260000,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1:8740000-8760000,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1:11240000-11260000,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1:13740000-13760000,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,19:53740000-53760000,989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
984,19:56240000-56260000,990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
985,19:58740000-58760000,991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
986,19:61240000-61260000,992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
max_bit = np.max([np.max([int(_b[1:]) for _b in _bits]) for _gname, _bits in region_2_readout_dict.items()]) + 1

In [38]:
for _gname, _bits in region_2_readout_dict.items():
    binary_code = []
    for _i in range(max_bit):
        if f"c{_i}" in _bits:
            binary_code.append(1)
        else:
            binary_code.append(0)
    print(_gname)
    codebook.loc[codebook['id']==_gname, codebook.columns[2:]] = binary_code
    #codebook.loc[codebook['name']==_gname,'id'] = int(_bits[0].split('u')[1])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
140
141
142
143
144
145
146
147
148
150
151
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
190
191
192
193
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
266
267
268
269
270
271
272
273
274
276
277
278
279
281
282
283
285
286
287
288
289
290
29

In [41]:
codebook

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_1028,NDB_925,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910
0,1:3740000-3760000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1:6240000-6260000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1:8740000-8760000,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1:11240000-11260000,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1:13740000-13760000,5,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,19:53740000-53760000,989,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
984,19:56240000-56260000,990,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
985,19:58740000-58760000,991,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
986,19:61240000-61260000,992,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [42]:
chr_2_region_num = pickle.load(open(os.path.join(library_folder, 'chr_2_region_num.pkl'), 'rb'))

In [43]:
chr_2_region_num

{'1': 77,
 '2': 70,
 '3': 61,
 '4': 59,
 '5': 58,
 '6': 58,
 '7': 49,
 '8': 50,
 '9': 48,
 '10': 51,
 '11': 48,
 '12': 43,
 '13': 45,
 '14': 42,
 '15': 40,
 '16': 38,
 '17': 36,
 '18': 35,
 '19': 23,
 'X': 56,
 'Y': 1}

In [45]:
chr_2_region_names = pickle.load(open(os.path.join(library_folder, 'chr_2_region_names.pkl'), 'rb'))

In [46]:
chr_2_region_names

{'1': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77],
 '2': [80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  150,
  151],
 '3': [153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  16

In [47]:
# generate region_to_chr_id
region_2_chr = {}
region_2_chr_order = {}
for _chr, _regions in chr_2_region_names.items():
    for _i, _region in enumerate(_regions):
        region_2_chr[_region] = _chr
        region_2_chr_order[_region] = _i

In [48]:
codebook['chr'] = [region_2_chr[_g] for _g in codebook['id']]
codebook['chr_order'] = [region_2_chr_order[_g] for _g in codebook['id']]

In [54]:
codebook.sort_values('id', inplace=True)

In [56]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-1000_codebook.csv')

In [57]:
codebook.to_csv(save_filename,
                index=None)

# generate 500 gene codebook

In [79]:
codebook_500 = pd.DataFrame(columns=codebook.columns)

# extract 500 genes
sel_regions_500 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_500':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_500:
            sel_regions_500.append(_gene)
            
for _gene in sel_regions_500:
    codebook_500 = codebook_500.append(codebook.loc[codebook['id']==int(_gene)])
codebook_500.sort_values('id', inplace=True)
codebook_500

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
0,1:3740000-3760000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0,1:3740000-3760000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1:8740000-8760000,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
2,1:8740000-8760000,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
4,1:13740000-13760000,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,X:161240000-161260000,1057,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,X,52
437,X:166240000-166260000,1059,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,X,54
437,X:166240000-166260000,1059,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,X,54
987,Y:1240000-1260000,1062,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,Y,0


In [80]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-500_codebook.csv')

codebook_500.to_csv(save_filename, index=None)

# 250-1

In [81]:
codebook_250_1 = pd.DataFrame(columns=codebook.columns)

# extract 250_1 genes
sel_regions_250_1 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_250_1':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_250_1:
            sel_regions_250_1.append(_gene)
            
for _gene in sel_regions_250_1:
    codebook_250_1 = codebook_250_1.append(codebook.loc[codebook['id']==int(_gene)])
codebook_250_1.sort_values('id', inplace=True)
codebook_250_1

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
1,1:6240000-6260000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
5,1:16240000-16260000,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,5
9,1:26240000-26260000,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,9
13,1:36240000-36260000,14,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,13
17,1:46240000-46260000,18,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,X:116240000-116260000,1039,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,37
424,X:131240000-131260000,1045,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,41
428,X:141240000-141260000,1049,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,45
432,X:153740000-153760000,1054,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,49


In [82]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-250_1_codebook.csv')

codebook_250_1.to_csv(save_filename, index=None)

# 250-2

In [83]:
codebook_250_2 = pd.DataFrame(columns=codebook.columns)

# extract 250_2 genes
sel_regions_250_2 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_250_2':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_250_2:
            sel_regions_250_2.append(_gene)
            
for _gene in sel_regions_250_2:
    codebook_250_2 = codebook_250_2.append(codebook.loc[codebook['id']==int(_gene)])
codebook_250_2.sort_values('id', inplace=True)
codebook_250_2

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
3,1:11240000-11260000,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,3
7,1:21240000-21260000,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,7
11,1:31240000-31260000,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,11
15,1:41240000-41260000,16,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,15
19,1:51240000-51260000,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,X:121240000-121260000,1041,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,X,39
426,X:136240000-136260000,1047,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,43
430,X:146240000-146260000,1051,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,X,47
434,X:158740000-158760000,1056,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,51


In [84]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-250_2_codebook.csv')

codebook_250_2.to_csv(save_filename, index=None)